### Бейзлайн модели

**Импорт библиотек**

Локально совсем не запускается, пробую колаб. Тогда `pip install` не трогать, которые закомменчены.

In [ ]:
# !pip install tensorflow_text

In [ ]:
# !pip install tf-keras==2.16.0 --no-dependencies

In [13]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [4]:
%%time

import pandas as pd

from transformers import AutoTokenizer

from datasets import Dataset, load_dataset
from huggingface_hub import login

from transformers import DataCollatorWithPadding

import evaluate
import numpy as np

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

CPU times: user 5 s, sys: 450 ms, total: 5.45 s
Wall time: 6.89 s


**Загружаем предобработанные данные**

In [48]:
df = pd.read_parquet('df.parquet')

In [49]:
dataset = df.rename(columns={'value': 'text', 'variable': 'label'}).drop(columns='prompt')

In [50]:
dataset.label = dataset.label.replace({'human_output': 0, 'model_output': 1, 'human_edits': 2})

/tmp/ipython-input-2489221204.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.label = dataset.label.replace({'human_output': 0, 'model_output': 1, 'human_edits': 2})


In [53]:
dataset = Dataset.from_pandas(dataset)

**Дообучаем `DeBERTa`**

In [9]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [56]:
def preprocess_function(example):
    text = example['text']
    labels = example['label']
    example = tokenizer(text, truncation=True, max_length=512)
    example['labels'] = labels

    return example

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/6561 [00:00<?, ? examples/s]

In [57]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [58]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = sigmoid(predictions)
    predictions = (predictions > 0.5).astype(int).reshape(-1)

    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [59]:
model = AutoModelForSequenceClassification.from_pretrained(

   'microsoft/deberta-v3-small', num_labels=3,
                       problem_type = "single_label_classification"
)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TODO. Что за ворнинг вылезает? Все запускается и по идее работает адекватно?

In [60]:
training_args = TrainingArguments(

   output_dir="my_awesome_model",
   learning_rate=2e-5,
   per_device_train_batch_size=3,
   per_device_eval_batch_size=3,
   num_train_epochs=2,
   weight_decay=0.01,
   eval_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset,
   eval_dataset=tokenized_dataset,  # TODO: как правильно подобрать eval датасет?
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-729631744.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 